In [31]:
# Load Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sci_optim
from sklearn.decomposition import PCA
from functions import *


In [32]:
# Load data

data = pd.read_csv('Data/non_nan_data.csv')
data = data.drop(['sun_mean', 'wind_mean', 'temp_mean', 'DK1_spot'], axis=1)

cols_to_lag = [col for col in data.columns if col.startswith('sun_') or col.startswith('temp_') or col.startswith('wind_')]
data[cols_to_lag] = data[cols_to_lag].shift(1)

data = data.dropna()

# Split data
data_train = data[(data['DATE'] >= '2014-01-01') & (data['DATE'] < '2023-01-01')].drop(['DATE'], axis=1).to_numpy()
data_test = data[data['DATE'] >= '2023-01-01'].drop(['DATE'], axis=1).to_numpy()

# To only use weather data one hour before the price, we lag weather data one hour
y_train = data_train[:, 0].reshape(-1, 1)
y_test = data_test[:, 0].reshape(-1, 1)

weather_train = data_train[:, 1:]
weather_test = data_test[:, 1:]

hour_dummies_train = sdummy(len(data_train), 24)
hour_dummies_test = sdummy(len(data_test), 24)

all_train = np.concatenate((weather_train, hour_dummies_train), axis=1)
all_test = np.concatenate((weather_test, hour_dummies_test), axis=1)

In [33]:
def perform_pca_and_residual_analysis(X, n_components):
    # Perform PCA
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)
    
    # Reconstruct the original dataset from the PCA-transformed data
    X_reconstructed = pca.inverse_transform(X_pca)
    
    # Calculate the residuals (difference between original and reconstructed data)
    residuals = X - X_reconstructed
    residual_error = np.linalg.norm(residuals, axis=1)  # Calculate the error norm for each sample
    
    # Return the residual error and the first n principal components
    return X_pca, residual_error

In [34]:
X_pca, residual_error = perform_pca_and_residual_analysis(weather_train, 100)



/Users/emilbisiach/miniconda3/envs/dev_env/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:611: RuntimeWarning: invalid value encountered in matmul
  C = X.T @ X


In [36]:
X_pca.shape

(78887, 100)